# Sustainability Report Parser

This file is created in order to create a parser that automatically extracts out the ESG data from tables in sustainability reports.

In [ ]:
%%capture

%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai
%pip install llama-index-core llama-parse
%pip install llama-index-core llama-parse llama-index-readers-file python-dotenv
%pip install llama-index-program-openai
%pip install -U llama-index llama-index-vector-stores-qdrant fastembed


OpenAI

In [ ]:
# If using OpenAI LLMs to get keywords
import os
import openai

os.environ["OPENAI_API_KEY"] =

from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode

llm = OpenAI(temperature=0.1, model="gpt-4o-mini", max_tokens=512)

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

EMBEDDING_MODEL  = "text-embedding-3-small"
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)

LLama parser

In [ ]:
# Uncomment if you are in a Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

#PLEASE RUN ONLY ONCE, 1k pages/day

from llama_parse import LlamaParse  # pip install llama-parse
from llama_index.core import SimpleDirectoryReader  # pip install llama-index

parse_instructions = '''This xlsx contains the ESG performace of Norilsk Nickels.  It contains many tables. Always ensure that a column representing the year of disclosure has a value in every row. If no values are found, replace it with NA. Answer questions using the tables found in this document and be precise.'''

parser = LlamaParse(
    api_key=,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    parsing_instruction= parse_instructions,
)

file_extractor = {".xlsx": parser}
reader = SimpleDirectoryReader("/content", file_extractor=file_extractor)
documents = reader.load_data()

ValueError: No files found in /content.

In [ ]:
from llama_index.core.node_parser import MarkdownElementNodeParser

# Parse the documents using MarkdownElementNodeParser
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8).from_defaults()

# Retrieve nodes (text) and objects (table)
nodes = node_parser.get_nodes_from_documents(documents)

base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

Pydantic model

In [ ]:
from pydantic import BaseModel
from typing import List

from llama_index.program.openai import OpenAIPydanticProgram


class Fact(BaseModel):
  """Sustainability Fact"""

  unit: str
  value: float
  year: int

class Metric(BaseModel):
  """Sustainability Metric"""

  values: List[Fact]

Vector Store

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings
from llama_index.vector_stores.qdrant import QdrantVectorStore
from google.colab import userdata
import qdrant_client

client = qdrant_client.QdrantClient(
    # otherwise set Qdrant instance address with:
    url= "https://af77da8c-504c-4c06-8140-407b89986d03.europe-west3-0.gcp.cloud.qdrant.io:6333",
    # set API KEY for Qdrant Cloud
    api_key= userdata.get('Qdrant'),
)

In [ ]:
# create our vector store with hybrid indexing enabled
# batch_size controls how many nodes are encoded with sparse vectors at once
vector_store = QdrantVectorStore(
    "Sustainability_Report_Parser_Test1",
    # client=client,
    # aclient=aclient,
    enable_hybrid=True,
    api_key = userdata.get('Qdrant'),
    url="https://af77da8c-504c-4c06-8140-407b89986d03.europe-west3-0.gcp.cloud.qdrant.io:6333"
)

#combine both into storage contect
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes = base_nodes + objects, storage_context=storage_context)


In [ ]:
objects

In [ ]:
# one extra dep
from llama_index.core import VectorStoreIndex

# create an index from the parsed markdown
#index = VectorStoreIndex.from_documents(documents)
index = VectorStoreIndex(
    nodes= base_nodes + objects
)

#create a query engine for the index

query_engine = index.as_query_engine(similarity_top_k = 2)


In [ ]:
prompt = """

Extract the number of strikes and lockouts in this report. This data can be found from a table.

Please provide a structure response with the following JSON schema:
{
  "values" : [Fact]
}

Where each fact has the following JSON schema:
{
  "unit": "string"
  "value": "float"
  "year": "integer"
}

If you cannot find the data, please return an empty dictionary in the form of {}. If any of the values are not present, please return None.

"""

In [ ]:
retriever = index.as_retriever()
retrieved_nodes = retriever.retrieve(prompt)

In [ ]:
print(retrieved_nodes)

In [ ]:
response = query_engine.query(prompt)

In [ ]:
print(response.response)

In [ ]:
import pandas as pd
import json

pd.DataFrame(json.loads(response.response)['values'])

In [ ]:
query_engine.get_prompts()

Define a prompt viewing function

In [ ]:
from IPython.display import Markdown, display

# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))

prompts_dict = query_engine.get_prompts()

display_prompt_dict(prompts_dict)

Prompts to use

In [ ]:
def make_prompt(metric):

  prompt = f"""

  Extract the {metric} in this report. This data can be found from a table.
  """ + """

  Please provide a structure response with the following JSON schema:
  {
    "values" : [Fact]
  }

  Where each fact has the following JSON schema:
  {
    "year": "integer"
    "value": "float"
    "unit": "string"
  }

  If you cannot find the data, please return an empty dictionary in the form of {}. If any of the values are not present, please return None.

  """

  return prompt

Reading in SASB standards

In [ ]:
indicators_base = pd.read_csv('Main Database - Indicator Names.csv')

In [ ]:
metric_list = [
    #GHG Emissions
    "total gross global scope 1 emissions",
    "total percentage of gross global scope 1 emissions under emissions-limiting regulations",
    #Air Quality
    "total CO emissions",
    "total NOx emissions",
    "total SOx emissions",
    "total PM10 emissions",
    "total mercury (HG) emissions",
    "total lead (PB) emissions",
    "total volatile organic compounds (VOCs) emissions",
    #Energy Management
    "total energy consumed",
    "total renewable energy consumed",
    "total energy consumed from grid electricity",
    "percentage energy consumed from grid electricity",
    "percentage energy consumed from renewable electricity",
    #Water Management
    "total freshwater withdrawn",
    "total freshwater consumed",
    "total water withdrawn in high or extremely high baseline water stress areas",
    "total water consumed in high or extremely high baseline water stress areas",
    "percentage of water withdrawn in high or extremely high baseline water stress areas against total freshwater withdrawn",
    "percentage of water consumed in high or extremely high baseline water stress areas against total freshwater consumed",
    "number of incidents of non-compliance associated with water quality permits, standards, and regulations",
    #Waste Management
    "total weight of tailing waste",
    "total weight of tailing waste recylced",
    "percentage of tailing waste recylced",
    "total weight of mineral waste",
    "total weight of mineral waste recylced",
    "percentage of mineral waste recylced",
    "number of tailing impoundments", #broken down by msha hazard potential <- need to add this
    #Biodiversity Impacts
    "number of mine sites where acid rock drainage is predicted to occur",
    "percentage of total mine sites where acid rock drainage is predicted to occur",
    "number of mine sites where acid rock drainage is actively mitigated",
    "percentage of total mine sites where acid rock drainage is actively mitigated",
    "number of mine sites where acid rock drainage is under treatment or remediation",
    "percentage of total mine sites where acid rock drainage is under treatment or remediation",
    "total area of proved reserves in or near sites with protected conservation status or endangered species habitat",
    "percentage of total area of proved reserves in or near sites with protected conservation status or endangered species habitat",
    "total area of probable reserves in or near sites with protected conservation status or endangered species habitat",
    "percentage of total area of probable reserves in or near sites with protected conservation status or endangered species habitat",
    #Human Rights
    "total area of proved reserves in or near areas of conflict",
    "percentage of total area of proved reserves in or near areas of conflict",
    "total area of probable reserves in or near areas of conflict",
    "percentage of total area of probable reserves in or near areas of conflict",
    "total area of proved reserves in or near indigenous land",
    "percentage of total area of proved reserves in or near indigenous land",
    "total area of probable reserves in or near indigenous land",
    "percentage of total area of probable reserves in or near indigenous land",
    #Community Relations
    "number of non-technical delays",
    "duration of non-technical delays",
    #Labour Relations
    "total number of employees covered under collective bargaining agreements",
    "total number of contractors covered under collective bargaining agreements",
    "percentage of employees covered under collective bargaining agreements",
    "percentage of contractors covered under collective bargaining agreements",
    "number of strikes and lockouts",
    "duration of strikes and lockouts",
    #Workforce Health and Safety
    "total number of MHSA incidents",
    "total MSHA all-incidents rate",
    "total number of fatalities",
    "total fatality rate",
    "total number of near misses",
    "total near miss frequecy rate (NMFR)",
    "average hours of health, safety and emergency response training for full time employees",
    "average hours of health, safety and emergency response training for contract employees",
    #Business Ethics and Transparency
    "production in countries that have the 20 lowest rankings in Transparency International's Corruption Perception Index",
    #Activity Metrics
    "total number of employees",
    "total number of contractors",
    "percentage of employees who are contractors",
]

In [ ]:
SASB_ind_dict = {
    #GHG Emissions
    "total gross global scope 1 emissions":"Gross global Scope 1 emissions, percentage covered under emissions-limiting regulations",
    "total percentage of gross global scope 1 emissions under emissions-limiting regulations":"Gross global Scope 1 emissions, percentage covered under emissions-limiting regulations",
    #Air Quality
    "total CO emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total NOx emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total SOx emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total PM10 emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total mercury (HG) emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total lead (PB) emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    "total volatile organic compounds (VOCs) emissions":"Air emissions of the following pollutants: (1) CO, (2) NOx (excluding N2O), (3) SOx, (4) particulate matter (PM10), (5) mercury (Hg), (6) lead (Pb), and (7) volatile organic compounds (VOCs)",
    #Energy Management
    "total energy consumed":"(1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable",
    "total renewable energy consumed":"(1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable",
    "total energy consumed from grid electricity":"(1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable",
    "percentage energy consumed from grid electricity":"(1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable",
    "percentage energy consumed from renewable electricity":"(1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable",
    #Water Management
    "total freshwater withdrawn":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "total freshwater consumed":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "total water withdrawn in high or extremely high baseline water stress areas":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "total water consumed in high or extremely high baseline water stress areas":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "percentage of water withdrawn in high or extremely high baseline water stress areas against total freshwater withdrawn":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "percentage of water consumed in high or extremely high baseline water stress areas against total freshwater consumed":"(1) Total fresh water withdrawn, (2) total fresh water consumed, percentage of each in regions with High or Extremely High Baseline Water Stress",
    "number of incidents of non-compliance associated with water quality permits, standards, and regulations":"Number of incidents of non-compliance associated with water quality permits, standards, and regulations",
    #Waste Management
    "total weight of tailing waste":"Total weight of tailings waste, percentage recycled",
    "total weight of tailing waste recylced":"Total weight of tailings waste, percentage recycled",
    "percentage of tailing waste recylced":"Total weight of tailings waste, percentage recycled",
    "total weight of mineral waste":"Total weight of mineral processing waste, percentage recycled",
    "total weight of mineral waste recylced":"Total weight of mineral processing waste, percentage recycled",
    "percentage of mineral waste recylced":"Total weight of mineral processing waste, percentage recycled",
    "number of tailing impoundments":"Number of tailings impoundments, broken down by MSHA hazard potential", #broken down by msha hazard potential <- need to add this
    #Biodiversity Impacts
    "number of mine sites where acid rock drainage is predicted to occur":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "percentage of total mine sites where acid rock drainage is predicted to occur":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "number of mine sites where acid rock drainage is actively mitigated":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "percentage of total mine sites where acid rock drainage is actively mitigated":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "number of mine sites where acid rock drainage is under treatment or remediation":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "percentage of total mine sites where acid rock drainage is under treatment or remediation":"Percentage of mine sites where acid rock drainage is: (1) predicted to occur, (2) actively mitigated, and (3) under treatment or remediation",
    "total area of proved reserves in or near sites with protected conservation status or endangered species habitat":"Percentage of (1) proved and (2) probable reserves in or near sites with protected conservation status or endangered species habitat",
    "percentage of total area of proved reserves in or near sites with protected conservation status or endangered species habitat":"Percentage of (1) proved and (2) probable reserves in or near sites with protected conservation status or endangered species habitat",
    "total area of probable reserves in or near sites with protected conservation status or endangered species habitat":"Percentage of (1) proved and (2) probable reserves in or near sites with protected conservation status or endangered species habitat",
    "percentage of total area of probable reserves in or near sites with protected conservation status or endangered species habitat":"Percentage of (1) proved and (2) probable reserves in or near sites with protected conservation status or endangered species habitat",
    #Human Rights
    "total area of proved reserves in or near areas of conflict":"Percentage of (1) proved and (2) probable reserves in or near areas of conflict",
    "percentage of total area of proved reserves in or near areas of conflict":"Percentage of (1) proved and (2) probable reserves in or near areas of conflict",
    "total area of probable reserves in or near areas of conflict":"Percentage of (1) proved and (2) probable reserves in or near areas of conflict",
    "percentage of total area of probable reserves in or near areas of conflict":"Percentage of (1) proved and (2) probable reserves in or near areas of conflict",
    "total area of proved reserves in or near indigenous land":"Percentage of (1) proved and (2) probable reserves in or near indigenous land",
    "percentage of total area of proved reserves in or near indigenous land":"Percentage of (1) proved and (2) probable reserves in or near indigenous land",
    "total area of probable reserves in or near indigenous land":"Percentage of (1) proved and (2) probable reserves in or near indigenous land",
    "percentage of total area of probable reserves in or near indigenous land":"Percentage of (1) proved and (2) probable reserves in or near indigenous land",
    #Community Relations
    "number of non-technical delays":"Number and duration of non-technical delays",
    "duration of non-technical delays":"Number and duration of non-technical delays",
    #Labour Relations
    "total number of employees covered under collective bargaining agreements":"Percentage of active workforce covered under collective bargaining agreements, broken down by U.S. and foreign employees",
    "total number of contractors covered under collective bargaining agreements":"Percentage of active workforce covered under collective bargaining agreements, broken down by U.S. and foreign employees",
    "percentage of employees covered under collective bargaining agreements":"Percentage of active workforce covered under collective bargaining agreements, broken down by U.S. and foreign employees",
    "percentage of contractors covered under collective bargaining agreements":"Percentage of active workforce covered under collective bargaining agreements, broken down by U.S. and foreign employees",
    "number of strikes and lockouts":"Number and duration of strikes and lockouts",
    "duration of strikes and lockouts":"Number and duration of strikes and lockouts",
    #Workforce Health and Safety
    "total number of MHSA incidents":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "total MSHA all-incidents rate":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "total number of fatalities":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "total fatality rate":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "total number of near misses":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "total near miss frequecy rate (NMFR)":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "average hours of health, safety and emergency response training for full time employees":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    "average hours of health, safety and emergency response training for contract employees":"(1) MSHA all-incidence rate, (2) fatality rate, (3) near miss frequency rate (NMFR) and (4) average hours of health, safety, and emergency response training for (a) full-time employees and (b) contract employees",
    #Business Ethics and Transparency
    "production in countries that have the 20 lowest rankings in Transparency International's Corruption Perception Index":"Production in countries that have the 20 lowest rankings in Transparency International’s Corruption Perception Index",
    #Activity Metrics
    "total number of employees":"Total number of employees, percentage contractors",
    "total number of contractors":"Total number of employees, percentage contractors",
    "percentage of employees who are contractors":"Total number of employees, percentage contractors",
}

In [ ]:
SASB_code_dict = {
    #GHG Emissions
    "total gross global scope 1 emissions":"EM-MM-110a.1",
    "total percentage of gross global scope 1 emissions under emissions-limiting regulations":"EM-MM-110a.1",
    #Air Quality
    "total CO emissions":"EM-MM-120a.1",
    "total NOx emissions":"EM-MM-120a.1",
    "total SOx emissions":"EM-MM-120a.1",
    "total PM10 emissions":"EM-MM-120a.1",
    "total mercury (HG) emissions":"EM-MM-120a.1",
    "total lead (PB) emissions":"EM-MM-120a.1",
    "total volatile organic compounds (VOCs) emissions":"EM-MM-120a.1",
    #Energy Management
    "total energy consumed":"EM-MM-130a.1",
    "total renewable energy consumed":"EM-MM-130a.1",
    "total energy consumed from grid electricity":"EM-MM-130a.1",
    "percentage energy consumed from grid electricity":"EM-MM-130a.1",
    "percentage energy consumed from renewable electricity":"EM-MM-130a.1",
    #Water Management
    "total freshwater withdrawn":"EM-MM-140a.1",
    "total freshwater consumed":"EM-MM-140a.1",
    "total water withdrawn in high or extremely high baseline water stress areas":"EM-MM-140a.1",
    "total water consumed in high or extremely high baseline water stress areas":"EM-MM-140a.1",
    "percentage of water withdrawn in high or extremely high baseline water stress areas against total freshwater withdrawn":"EM-MM-140a.1",
    "percentage of water consumed in high or extremely high baseline water stress areas against total freshwater consumed":"EM-MM-140a.1",
    "number of incidents of non-compliance associated with water quality permits, standards, and regulations":"EM-MM-140a.2",
    #Waste Management
    "total weight of tailing waste":"EM-MM-150a.1",
    "total weight of tailing waste recylced":"EM-MM-150a.1",
    "percentage of tailing waste recylced":"EM-MM-150a.1",
    "total weight of mineral waste":"EM-MM-150a.2",
    "total weight of mineral waste recylced":"EM-MM-150a.2",
    "percentage of mineral waste recylced":"EM-MM-150a.2",
    "number of tailing impoundments":"EM-MM-150a.3", #broken down by msha hazard potential <- need to add this
    #Biodiversity Impacts
    "number of mine sites where acid rock drainage is predicted to occur":"EM-MM-160a.2",
    "percentage of total mine sites where acid rock drainage is predicted to occur":"EM-MM-160a.2",
    "number of mine sites where acid rock drainage is actively mitigated":"EM-MM-160a.2",
    "percentage of total mine sites where acid rock drainage is actively mitigated":"EM-MM-160a.2",
    "number of mine sites where acid rock drainage is under treatment or remediation":"EM-MM-160a.2",
    "percentage of total mine sites where acid rock drainage is under treatment or remediation":"EM-MM-160a.2",
    "total area of proved reserves in or near sites with protected conservation status or endangered species habitat":"EM-MM-160a.3",
    "percentage of total area of proved reserves in or near sites with protected conservation status or endangered species habitat":"EM-MM-160a.3",
    "total area of probable reserves in or near sites with protected conservation status or endangered species habitat":"EM-MM-160a.3",
    "percentage of total area of probable reserves in or near sites with protected conservation status or endangered species habitat":"EM-MM-160a.3",
    #Human Rights
    "total area of proved reserves in or near areas of conflict":"EM-MM-210a.1",
    "percentage of total area of proved reserves in or near areas of conflict":"EM-MM-210a.1",
    "total area of probable reserves in or near areas of conflict":"EM-MM-210a.1",
    "percentage of total area of probable reserves in or near areas of conflict":"EM-MM-210a.1",
    "total area of proved reserves in or near indigenous land":"EM-MM-210a.2",
    "percentage of total area of proved reserves in or near indigenous land":"EM-MM-210a.2",
    "total area of probable reserves in or near indigenous land":"EM-MM-210a.2",
    "percentage of total area of probable reserves in or near indigenous land":"EM-MM-210a.2",
    #Community Relations
    "number of non-technical delays":"EM-MM-210b.2",
    "duration of non-technical delays":"EM-MM-210b.2",
    #Labour Relations
    "total number of employees covered under collective bargaining agreements":"EM-MM-310a.1",
    "total number of contractors covered under collective bargaining agreements":"EM-MM-310a.1",
    "percentage of employees covered under collective bargaining agreements":"EM-MM-310a.1",
    "percentage of contractors covered under collective bargaining agreements":"EM-MM-310a.1",
    "number of strikes and lockouts":"EM-MM-310a.2",
    "duration of strikes and lockouts":"EM-MM-310a.2",
    #Workforce Health and Safety
    "total number of MHSA incidents":"EM-MM-320a.1",
    "total MSHA all-incidents rate":"EM-MM-320a.1",
    "total number of fatalities":"EM-MM-320a.1",
    "total fatality rate":"EM-MM-320a.1",
    "total number of near misses":"EM-MM-320a.1",
    "total near miss frequecy rate (NMFR)":"EM-MM-320a.1",
    "average hours of health, safety and emergency response training for full time employees":"EM-MM-320a.1",
    "average hours of health, safety and emergency response training for contract employees":"EM-MM-320a.1",
    #Business Ethics and Transparency
    "production in countries that have the 20 lowest rankings in Transparency International's Corruption Perception Index":"EM-MM-510a.2",
    #Activity Metrics
    "total number of employees":"EM-MM-000.B",
    "total number of contractors":"EM-MM-000.B",
    "percentage of employees who are contractors":"EM-MM-000.B",
}

In [ ]:
def extracted_info(metric):
  query = make_prompt(metric)
  response = query_engine.query(query)
  return response.response


In [ ]:
from tqdm import tqdm
import concurrent.futures

# Using ThreadPoolExecutor to create a list in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    # Map the function to the inputs in parallel
    results = list(tqdm(executor.map(extracted_info, metric_list)))

In [ ]:
print(results[0])

In [ ]:
def write_df(results):
  df = pd.DataFrame()
  counter = 0

  for result in results:
    try:
      result_df = pd.DataFrame(json.loads(result)['values'])
    except:
      pass
    else:
      num_entries = len(result_df)

      if num_entries > 0:
        metric = metric_list[counter]
        result_df.insert(3, 'remarks', metric)
        result_df.insert(0, 'indicator', SASB_code_dict[metric])
        result_df.insert(1, 'indicator name', SASB_ind_dict[metric])
        #result_df['remark'] = [metric] * num_entries
        df = pd.concat([df,result_df], ignore_index=True)
    counter += 1
  return df

In [ ]:
df = write_df(results)

df

In [ ]:
df.to_csv('Norilsk Nickels data.csv', index = False)

Lets try pure chatgpt wout llama index

In [ ]:
from openai import OpenAI

client = OpenAI()

#When Creating the assistant for the first timme
assistant = client.beta.assistants.create(
  name="Albemarle Data Scraper",
  instructions=
  """You are an ESG data scraper for Albemarle, a chemical manufacturing company.
  You will be provided a pdf file which contains many ESG tables. You will be tasked to extract out ESG information.

  Please provide a structured response with the following JSON schema:
  {
    "values" : [Fact]
  }

  Where each fact has the following JSON schema:
  {
    "year": "integer"
    "value": "float"
    "unit": "string"
  }

  If you cannot find the data, please return an empty dictionary in the form of {}. If any of the values are not present, please return NA for that attribute.

  """,
  model="gpt-4o-mini",
  tools=[{"type": "file_search"}],
)

#assistant = client.beta.assistants.retrieve('asst_pbID5ew1pwi2PuqLkuLljS47')

In [ ]:
# Create a vector store caled "{Company} ESG Statements"
vector_store = client.beta.vector_stores.create(name="Albermarle Databook")

# Ready the files for upload to OpenAI
file_paths = ["/content/Albemarle.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

In [ ]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("/content/Cameco Databook.pdf", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Extract the gross global scope 1 emissions in this report.",
      # Attach the new file to the message.
      "attachments": [
        {"file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    },
  ]
)

# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI

# client = OpenAI()

class EventHandler(AssistantEventHandler):

    # def on_text_created(self, text) -> None:
    #     print(f"\nassistant > ", end="", flush=True)

    # def on_tool_call_created(self, tool_call):
    #     print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        #print(message_content.value)
        #print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

# with client.beta.threads.runs.stream(
#     thread_id=thread.id,
#     assistant_id=assistant.id,
#     event_handler=EventHandler(),
# ) as stream:
#     stream.until_done()

In [ ]:
print(openai.beta.threads.messages.list(thread.id).data[0].content[0].text.value)

In [ ]:
txt = openai.beta.threads.messages.list(thread.id).data[0].content[0].text.value
print(txt.split("```json")[1].split("```")[0])

In [ ]:
def extract_info(prompt, assistant):

  # assistant = client.beta.assistants.retrieve('asst_jlqE279HiI2fagr9Cg1wKUrO')

  # vector_store = client.beta.vector_stores.retrieve('vs_33fCZa1x5FlcKr32bnjpOT6W')

  # assistant = client.beta.assistants.update(
  # assistant_id=assistant.id,
  # tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
  # )

  # Create a thread and attach the file to the message
  thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": f"Extract the {prompt} in this report.",
        # Attach the new file to the message.
      },
    ]
  )
  # assistant = assistant

  with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
  ) as stream:
    stream.until_done()

  txt = openai.beta.threads.messages.list(thread.id).data[0].content[0].text.value
  #print(openai.beta.threads.messages.list(thread.id))
  return(txt)


In [ ]:
out = extract_info("the gross global scope 1 emissions")
print(out)

In [ ]:
out.split("```json")[1].split("```")[0]

In [ ]:
import concurrent.futures
import time
from tqdm import tqdm
# results = []

# for metric in metric_list:
#   results.append(extract_info(metric))



# Using ThreadPoolExecutor to create a list in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    # Map the function to the inputs in parallel
    results = list(tqdm(executor.map(extract_info, metric_list)))


In [ ]:
def get_json(outs):
  final = []
  for out in outs:
    try:
      split = out.split("```json")[1].split("```")[0]
      final.append(split)
    except:
      final.append(out)
  return final

def write_df(results):

  import pandas as pd
  import json

  df = pd.DataFrame()
  counter = 0

  for result in results:
    try:
      result_df = pd.DataFrame(json.loads(result)['values'])
    except:
      pass
    else:
      num_entries = len(result_df)

      if num_entries > 0:
        metric = metric_list[counter]
        result_df.insert(3, 'remarks', metric)
        result_df.insert(0, 'indicator', SASB_code_dict[metric])
        result_df.insert(1, 'indicator name', SASB_ind_dict[metric])
        #result_df['remark'] = [metric] * num_entries
        df = pd.concat([df,result_df], ignore_index=True)
    counter += 1
  return df

In [ ]:
import pandas as pd
import json
df = write_df(get_json(results))

df.to_csv('Albemarle data 2.csv', index = False)

## Putting it all in 1 chunk for easier loading.

In [ ]:
def parse_doc(company, company_desc, sus_report):

  from openai import OpenAI

  client = OpenAI()

  #Create Assistant to scrape Data
  assistant = client.beta.assistants.create(
    name=f"{company} Data Scraper",
    instructions=
    f"""You are an ESG data scraper for {company}, a {company_desc}."""
     +
    """You will be provided a pdf file which contains many ESG tables. You will be tasked to extract out ESG information.

    Please provide a structured response with the following JSON schema:
    {
      "values" : [Fact]
    }

    Where each fact has the following JSON schema:
    {
      "year": "integer"
      "value": "float"
      "unit": "string"
    }

    If you cannot find the data, please return an empty dictionary in the form of {}. If any of the values are not present, please return NA for that attribute.

    """,
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
  )


  # Create a vector store caled "{Company} Databook"
  vector_store = client.beta.vector_stores.create(name=f"{company} Databook")

  # Ready the files for upload to OpenAI
  file_paths = [sus_report]
  file_streams = [open(path, "rb") for path in file_paths]

  # Use the upload and poll SDK helper to upload the files, add them to the vector store,
  # and poll the status of the file batch for completion.
  file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
  )

  # You can print the status and the file counts of the batch to see the result of this operation.
  # print(file_batch.status)
  # print(file_batch.file_counts)

  #Attach vector store to assistant
  assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
  )


  import concurrent.futures
  import time
  from tqdm import tqdm

  ass_list = [assistant] * len(metric_list)

  # Using ThreadPoolExecutor to create a list in parallel
  with concurrent.futures.ProcessPoolExecutor() as executor:
      # Map the function to the inputs in parallel
      results = list(tqdm(executor.map(extract_info, metric_list, ass_list)))

  df = write_df(get_json(results))

  df.to_csv(f'{company} data.csv', index = False)


In [ ]:
parse_doc("ComfortDelGro","multi-national transport group", "/content/ComfortDelGro Sustainability Report 2023.pdf")